___

# Etude sur la malnutrition basée sur les données de la FAO

Source des données : http://www.fao.org/faostat/en/#home

___

# Document A
## ✓ Import des données de la FAO
## ✓ Traitement et nettoyage
## ✓ Analyse

___

# Part 0

## Objectifs

- Import des librairies et des données
- Création des dataframes principaux

___

## Méthode

J’ai d’abord exporté les 5 tables suivantes depuis le site de la FAO :

-	Bilans alimentaires – Produits d’origine animale
-	Bilans alimentaires – Produits d’origine végétale
-	Bilans alimentaires – Population par pays
-	Bilans alimentaires – Liste des céréales dans le monde
-	Sécurité alimentaire – Personnes sous-alimentées par pays

J’ai ensuite créé un dataframe par jeu de données.

Après avoir créé une colonne pour identifier les produits d'origine animale et une colonne pour les produits d'origine végétale, j'ai concaténé les deux dans un dataframe principal.

Enfin, j'ai pivoté la colonne "Elements" pour avoir une seule ligne par trio "Année / Pays / Produit".

___

## Code et résultats

In [1]:
# Libraries import

import random
import numpy as np
import pandas as pd
from IPython.display import display

# Configuration

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 1000)

# Global variables

# kcal / capita / year
cal_need = 2200 * 365

# kg / capita / year
prot_need = 62 * 0.8 * 365 / 1000

# Actuel humans
humans = 7530000000

In [2]:
# Funtions useful for the project

def to_int(x):
    return int(round(x,0))

def to_str(x):
    return str(int(round(x,0)))

In [3]:
# "Bilan alimentaire" columns
ba_columns = ['Code Pays', 'Pays', 'Élément', 'Code Produit', 'Produit', 'Année', 'Valeur']

# "Sécurité alimentaire" columns
sa_columns = ['Code zone', 'Zone', 'Élément', 'Produit', 'Année', 'Valeur']

In [4]:
# Food balance for animal products
df_animal_products = pd.read_csv("data/animal_products.csv")[ba_columns]
df_animal_products['food_origin'] = 'animal'
df_animal_products.head(5)

,Code Pays,Pays,Élément,Code Produit,Produit,Année,Valeur,food_origin
0,2,Afghanistan,Production,2731,Viande de Bovins,2013,134.00,animal
1,2,Afghanistan,Importations - Quantité,2731,Viande de Bovins,2013,6.00,animal
2,2,Afghanistan,Disponibilité intérieure,2731,Viande de Bovins,2013,140.00,animal
3,2,Afghanistan,Nourriture,2731,Viande de Bovins,2013,140.00,animal
4,2,Afghanistan,Disponibilité alimentaire en quantité (kg/pers...,2731,Viande de Bovins,2013,4.59,animal


In [5]:
# Food balance for vegetable products
df_vegetable_products = pd.read_csv("data/vegetable_products.csv")[ba_columns]
df_vegetable_products['food_origin'] = 'vegetable'
df_vegetable_products.head(5)

,Code Pays,Pays,Élément,Code Produit,Produit,Année,Valeur,food_origin
0,2,Afghanistan,Production,2511,Blé,2013,5169.0,vegetable
1,2,Afghanistan,Importations - Quantité,2511,Blé,2013,1173.0,vegetable
2,2,Afghanistan,Variation de stock,2511,Blé,2013,-350.0,vegetable
3,2,Afghanistan,Disponibilité intérieure,2511,Blé,2013,5992.0,vegetable
4,2,Afghanistan,Semences,2511,Blé,2013,322.0,vegetable


In [6]:
# World population by country
df_pop = pd.read_csv("data/population.csv")[['Pays', 'Valeur']]
df_pop.columns = ['country', 'pop_size']

# Clean and scale
df_pop['pop_size'] = df_pop['pop_size'] * 1000
df_pop.fillna(0, inplace=True)

df_pop.head(5)

,country,pop_size
0,Afghanistan,30552000
1,Afrique du Sud,52776000
2,Albanie,3173000
3,Algérie,39208000
4,Allemagne,82727000


In [7]:
# Worl undernourishment by country
df_undernourishment = pd.read_csv("data/undernourishment.csv")[sa_columns]

df_undernourishment.rename(
    columns={
        'Code zone': 'country_code',
        'Zone': 'country',
        'Valeur': 'undernourishment',
        'Année': 'year'
    },
    inplace=True
)

def year_format(year_code):
    """ From year_code format (XXXXXXX) to standard year format (XXXX) """
    
    if year_code == '2011-2013':
        return 2012

    elif year_code == '2012-2014':
        return 2013
    
    elif year_code == '2013-2015':
        return 2014
    
    elif year_code == '2014-2016':
        return 2015
    
    elif year_code == '2015-2017':
        return 2016
    
    elif year_code == '2016-2018':
        return 2017

# Apply the standard year format XXXX
df_undernourishment['year'] = df_undernourishment['year'].apply(year_format)

df_undernourishment.fillna(0, inplace=True)
df_undernourishment['undernourishment'] = df_undernourishment['undernourishment'] * 1000000
df_undernourishment['undernourishment'] = df_undernourishment['undernourishment'].astype(int)
df_undernourishment['is_undernourishment'] = False
df_undernourishment.loc[df_undernourishment['undernourishment'] > 0, 'is_undernourishment'] = True

# Remove "China" (We just keep the China sub-countries)
df_undernourishment = df_undernourishment[df_undernourishment.country != 'Chine'].copy()

df_undernourishment.head(5)

,country_code,country,Élément,Produit,year,undernourishment,is_undernourishment
0,2,Afghanistan,Valeur,Nombre de personnes sous-alimentées (millions)...,2013,8100000,True
1,202,Afrique du Sud,Valeur,Nombre de personnes sous-alimentées (millions)...,2013,2600000,True
2,3,Albanie,Valeur,Nombre de personnes sous-alimentées (millions)...,2013,200000,True
3,4,Algérie,Valeur,Nombre de personnes sous-alimentées (millions)...,2013,1800000,True
4,79,Allemagne,Valeur,Nombre de personnes sous-alimentées (millions)...,2013,0,False


In [8]:
# Products list - Cereals only

df_cereals = pd.read_csv("data/cereals.csv")[ba_columns]

df_cereals.rename(columns={
    'Code Pays': 'country_code',
    'Pays': 'country',
    'Code Produit': 'product_code',
    'Produit': 'product',
    'Année': 'year',
}, inplace=True)

df_cereals['is_cereal'] = True

# Remove "China" (We just keep the China sub-countries)
df_cereals = df_cereals[df_cereals.country != 'Chine'].copy()

display(df_cereals.head(5))

,country_code,country,Élément,product_code,product,year,Valeur,is_cereal
0,5000,Monde,Production,2511,Blé,2013,708443.0,True
1,5000,Monde,Importations - Quantité,2511,Blé,2013,185378.0,True
2,5000,Monde,Variation de stock,2511,Blé,2013,-15947.0,True
3,5000,Monde,Exportations - Quantité,2511,Blé,2013,198378.0,True
4,5000,Monde,Disponibilité intérieure,2511,Blé,2013,679495.0,True


In [9]:
# Concatenate the "products" dataframes
df_list = ([df_vegetable_products, df_animal_products])
main_df = pd.concat(df_list)
main_df = main_df.reset_index(drop=True)

# Pivot table
main_df = pd.pivot_table(main_df, values='Valeur', index=['Code Pays', 'Pays', 'Code Produit', 'Produit', 'Année', 'food_origin'], columns='Élément', aggfunc=np.sum)
main_df = main_df.reset_index()

# Rename
main_df.rename(columns={
    'Code Pays': 'country_code',
    'Pays': 'country',
    'Code Produit': 'product_code',
    'Produit': 'product',
    'Année': 'year',
    'Aliments pour animaux': 'feed_kg',
    'Autres Utilisations': 'other_uses_kg',
    'Disponibilité alimentaire (Kcal/personne/jour)': 'food_supply_kcal_capita_year',
    'Disponibilité alimentaire en quantité (kg/personne/an)': 'food_supply_kg_capita_year',
    'Disponibilité de matière grasse en quantité (g/personne/jour)': 'fat_supply_kg_capita_year',
    'Disponibilité de protéines en quantité (g/personne/jour)': 'protein_supply_kg_capita_year',
    'Exportations - Quantité': 'export_quantity_kg',
    'Importations - Quantité': 'import_quantity_kg',
    'Nourriture': 'food_kg',
    'Pertes': 'losses_kg',
    'Production': 'production_kg',
    'Semences': 'seed_kg',
    'Traitement': 'processing_kg',
    'Variation de stock': 'stock_variation_kg',
    'Disponibilité intérieure': 'domestic_supply_kg'
}, inplace=True)

# Left join with the population size
main_df = main_df.merge(df_pop, how='left', on='country')

# Clean
main_df.fillna(0, inplace=True)

# From 1000 tons to kg
main_df['export_quantity_kg'] = main_df['export_quantity_kg'] * 1000000
main_df['other_uses_kg'] = main_df['other_uses_kg'] * 1000000
main_df['feed_kg'] = main_df['feed_kg'] * 1000000
main_df['import_quantity_kg'] = main_df['import_quantity_kg'] * 1000000
main_df['food_kg'] = main_df['food_kg'] * 1000000
main_df['losses_kg'] = main_df['losses_kg'] * 1000000
main_df['production_kg'] = main_df['production_kg'] * 1000000
main_df['seed_kg'] = main_df['seed_kg'] * 1000000
main_df['processing_kg'] = main_df['processing_kg'] * 1000000
main_df['stock_variation_kg'] = main_df['stock_variation_kg'] * 1000000
main_df['domestic_supply_kg'] = main_df['domestic_supply_kg'] * 1000000

# From per day to per year
main_df['food_supply_kcal_capita_year'] = main_df['food_supply_kcal_capita_year'] * 365

# From g / capita / day to kg / capita / year
main_df['fat_supply_kg_capita_year'] = main_df['fat_supply_kg_capita_year'] / 1000 * 365
main_df['protein_supply_kg_capita_year'] = main_df['protein_supply_kg_capita_year'] / 1000 * 365

# From float to int
main_df['feed_kg'] = main_df['feed_kg'].astype(int)
main_df['other_uses_kg'] = main_df['other_uses_kg'].astype(int)
main_df['food_supply_kcal_capita_year'] = main_df['food_supply_kcal_capita_year'].astype(int)
main_df['domestic_supply_kg'] = main_df['domestic_supply_kg'].astype(int)
main_df['export_quantity_kg'] = main_df['export_quantity_kg'].astype(int)
main_df['import_quantity_kg'] = main_df['import_quantity_kg'].astype(int)
main_df['food_kg'] = main_df['food_kg'].astype(int)
main_df['losses_kg'] = main_df['losses_kg'].astype(int)
main_df['production_kg'] = main_df['production_kg'].astype(int)
main_df['seed_kg'] = main_df['seed_kg'].astype(int)
main_df['processing_kg'] = main_df['processing_kg'].astype(int)
main_df['stock_variation_kg'] = main_df['stock_variation_kg'].astype(int)
main_df['pop_size'] = main_df['pop_size'].astype(int)

main_df.head(10)

,country_code,country,product_code,product,year,food_origin,feed_kg,other_uses_kg,food_supply_kcal_capita_year,food_supply_kg_capita_year,fat_supply_kg_capita_year,protein_supply_kg_capita_year,domestic_supply_kg,export_quantity_kg,import_quantity_kg,food_kg,losses_kg,production_kg,seed_kg,processing_kg,stock_variation_kg,pop_size
0,1,Arménie,2511,Blé,2013,vegetable,93000000,0,373760,130.60,1.31400,11.13980,554000000,1000000,361000000,389000000,32000000,312000000,30000000,10000000,-118000000,2977000
1,1,Arménie,2513,Orge,2013,vegetable,137000000,26000000,0,0.00,0.00000,0.00000,198000000,0,9000000,0,15000000,189000000,14000000,7000000,0,2977000
2,1,Arménie,2514,Maïs,2013,vegetable,96000000,0,0,0.03,0.00000,0.00365,102000000,0,82000000,0,7000000,21000000,0,0,0,2977000
3,1,Arménie,2515,Seigle,2013,vegetable,1000000,0,365,0.12,0.00000,0.00730,1000000,0,0,0,0,1000000,0,0,0,2977000
4,1,Arménie,2516,Avoine,2013,vegetable,4000000,0,730,0.37,0.01095,0.03285,6000000,0,1000000,1000000,0,5000000,0,0,0,2977000
5,1,Arménie,2517,Millet,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000
6,1,Arménie,2518,Sorgho,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000
7,1,Arménie,2520,"Céréales, Autres",2013,vegetable,1000000,12000000,5110,1.96,0.01460,0.14235,21000000,0,4000000,6000000,1000000,17000000,1000000,0,0,2977000
8,1,Arménie,2531,Pommes de Terre,2013,vegetable,225000000,16000000,29930,45.10,0.04380,0.70810,672000000,9000000,5000000,134000000,96000000,661000000,123000000,78000000,15000000,2977000
9,1,Arménie,2532,Manioc,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000


In [10]:
# Add the column "is_cereal" to the main DF

temp_df = pd.DataFrame(df_cereals, columns=['product_code', 'is_cereal'])
main_df = main_df.merge(temp_df, how='left', on='product_code')
main_df.drop_duplicates(inplace=True) # 'Code produit' appears several times in df_cereals
main_df['is_cereal'].fillna(False, inplace=True)

# Add the column "is_undernourishment" to the main DF

temp_df = pd.DataFrame(df_undernourishment, columns=['country_code', 'undernourishment', 'is_undernourishment'])
main_df = main_df.merge(temp_df, how='left', on='country_code')

# Remove "China" (We just keep the China sub-countries)
main_df = main_df[main_df.country != 'Chine'].copy()

display(main_df.head(10))

,country_code,country,product_code,product,year,food_origin,feed_kg,other_uses_kg,food_supply_kcal_capita_year,food_supply_kg_capita_year,fat_supply_kg_capita_year,protein_supply_kg_capita_year,domestic_supply_kg,export_quantity_kg,import_quantity_kg,food_kg,losses_kg,production_kg,seed_kg,processing_kg,stock_variation_kg,pop_size,is_cereal,undernourishment,is_undernourishment
0,1,Arménie,2511,Blé,2013,vegetable,93000000,0,373760,130.60,1.31400,11.13980,554000000,1000000,361000000,389000000,32000000,312000000,30000000,10000000,-118000000,2977000,True,100000.0,True
1,1,Arménie,2513,Orge,2013,vegetable,137000000,26000000,0,0.00,0.00000,0.00000,198000000,0,9000000,0,15000000,189000000,14000000,7000000,0,2977000,True,100000.0,True
2,1,Arménie,2514,Maïs,2013,vegetable,96000000,0,0,0.03,0.00000,0.00365,102000000,0,82000000,0,7000000,21000000,0,0,0,2977000,True,100000.0,True
3,1,Arménie,2515,Seigle,2013,vegetable,1000000,0,365,0.12,0.00000,0.00730,1000000,0,0,0,0,1000000,0,0,0,2977000,True,100000.0,True
4,1,Arménie,2516,Avoine,2013,vegetable,4000000,0,730,0.37,0.01095,0.03285,6000000,0,1000000,1000000,0,5000000,0,0,0,2977000,True,100000.0,True
5,1,Arménie,2517,Millet,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000,True,100000.0,True
6,1,Arménie,2518,Sorgho,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000,True,100000.0,True
7,1,Arménie,2520,"Céréales, Autres",2013,vegetable,1000000,12000000,5110,1.96,0.01460,0.14235,21000000,0,4000000,6000000,1000000,17000000,1000000,0,0,2977000,True,100000.0,True
8,1,Arménie,2531,Pommes de Terre,2013,vegetable,225000000,16000000,29930,45.10,0.04380,0.70810,672000000,9000000,5000000,134000000,96000000,661000000,123000000,78000000,15000000,2977000,False,100000.0,True
9,1,Arménie,2532,Manioc,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000,False,100000.0,True


___

# Part 1

## Objectifs

- Analyser et calculer la population mondiale.

___

## Méthode

On constate qu'il y a une redondance sur la Chine :

RAS de Hong-Kong + RAS de Macao + Chine continentale + Province de Taiwan = Chine

J'ai donc supprimé **Chine** pour garder uniquement les subdivisions.

___

## Code et résultats

In [11]:
pop_size = df_pop['pop_size'].sum()
pop_diff = pop_size / humans - 1

print(f"""2013 World population: {pop_size:,}
Actual World population: {humans:,}
Gap : +{pop_diff:.1%}
""")

display(df_pop[df_pop['country'].str.contains('hine')])

# Delete All "China sub-country"
try:
    df_pop = df_pop[df_pop.country != 'Chine'].copy()
    pop_size = df_pop['pop_size'].sum()
except:
    #Already done
    pass

print(f"""
2013 Fixed World population: {pop_size:,}
""")

2013 World population: 8,413,993,000
Actual World population: 7,530,000,000
Gap : +11.7%



,country,pop_size
33,Chine - RAS de Hong-Kong,7204000
34,Chine - RAS de Macao,566000
35,"Chine, continentale",1385567000
36,"Chine, Taiwan Province de",23330000
174,Chine,1416667000



2013 Fixed World population: 6,997,326,000



## Commentaires

Il fallait soit supprimer les subdivisons de la Chine, soit supprimer l'élément 'Chine'. J'ai supprimé l'élément 'Chine' afin d'avoir une granularité plus fine sur une région qui compte plus d'un milliard d'habitants.

___

# Part 2

## Objectifs

Identification des redondances parmi les variables suivantes :
    
- Production (Production)
- Importations (Import Quantity)
- Exportations (Export Quantity)
- Variation de stock (Stock Variation)
- Disponibilité intérieure (Domestic supply)
- Semences (Seed)
- Pertes (Waste)
- Nourriture (Food) , aussi appelée Disponibilité alimentaire (Food Supply)
- Aliments pour animaux (Feed)
- Traitement (Processing)
- Autres utilisations (Other uses)

___

## Méthode

J’ai analysé les différentes variables issues de "Elements", obtenues après pivot pour comprendre quelles étaient les informations redondantes. J’ai pu confirmer mon hypothèse en la testant sur les données du blé.

___


## Code et résultats

In [12]:
print("""
Term #1 : Productions + Importations - Exportations + Stock Variations
Term #2 : Domestic Supply
Term #3 : Seed + Food Supply + Feed + Processing + Other Uses + Waste
""")

fr_df = main_df[main_df['country'] == 'France']
ble_fr_df = fr_df[fr_df['product'] == 'Blé']

production = ble_fr_df['production_kg'].values[0]
importations = ble_fr_df['import_quantity_kg'].values[0]
exportations = ble_fr_df['export_quantity_kg'].values[0]
stock_variation = ble_fr_df['stock_variation_kg'].values[0]
domestic_supply = ble_fr_df['domestic_supply_kg'].values[0]
seed = ble_fr_df['seed_kg'].values[0]
waste = ble_fr_df['losses_kg'].values[0]
food = ble_fr_df['food_kg'].values[0]
feed = ble_fr_df['feed_kg'].values[0]
processing = ble_fr_df['processing_kg'].values[0]
other_uses = ble_fr_df['other_uses_kg'].values[0]

term1 = production + importations - exportations + stock_variation
term2 = domestic_supply
term3 = seed + food + feed + processing + other_uses + waste

print(f"""
Wheat data:

Production : {production}
Importations : {importations}
Exportations : {exportations}
Stock Variation : {stock_variation}
Domestic Supply : {domestic_supply}
Seed : {seed}
Waste : {waste}
Food : {food}
Feed : {feed}
Processing : {processing}
Other Uses : {other_uses}

production + importations - exportations + stock_variation = {term1}
domestic_supply = {term2}
seed + food + feed + processing + other_uses + waste = {term3}
""")


Term #1 : Productions + Importations - Exportations + Stock Variations
Term #2 : Domestic Supply
Term #3 : Seed + Food Supply + Feed + Processing + Other Uses + Waste


Wheat data:

Production : 38614000000
Importations : 2055000000
Exportations : 21502000000
Stock Variation : 1131000000
Domestic Supply : 20298000000
Seed : 748000000
Waste : 358000000
Food : 6971000000
Feed : 7822000000
Processing : 1575000000
Other Uses : 2824000000

production + importations - exportations + stock_variation = 20298000000
domestic_supply = 20298000000
seed + food + feed + processing + other_uses + waste = 20298000000




## Commentaires

Il y a en effet une redondance. 'Domestic Supply' étant une variable utile dans les analyses, il est judicieux qu'elle soit d'emblée intégrée. On peut retrouver sur le site de la FAO les définitions exactes de chaque élément.

___

# Part 3

## Objectifs

Calcul de la disponibilité alimentaire en kcal puis en kg de protéines

___

## Méthode

Pour créer les totaux « Food Supply » et « Protein Supply » j’ai multiplié les valeurs pour chaque produit par le nombre d’habitants et par 365 jours.

___


## Code ⇩

In [13]:
main_df['total_food_supply'] = main_df['food_supply_kcal_capita_year'] * main_df['pop_size']
main_df['total_protein_supply'] = main_df['protein_supply_kg_capita_year'] * main_df['pop_size']

main_df['total_food_supply'].fillna(0, inplace=True)
main_df['total_protein_supply'].fillna(0, inplace=True)

display(main_df.head(10))

,country_code,country,product_code,product,year,food_origin,feed_kg,other_uses_kg,food_supply_kcal_capita_year,food_supply_kg_capita_year,fat_supply_kg_capita_year,protein_supply_kg_capita_year,domestic_supply_kg,export_quantity_kg,import_quantity_kg,food_kg,losses_kg,production_kg,seed_kg,processing_kg,stock_variation_kg,pop_size,is_cereal,undernourishment,is_undernourishment,total_food_supply,total_protein_supply
0,1,Arménie,2511,Blé,2013,vegetable,93000000,0,373760,130.60,1.31400,11.13980,554000000,1000000,361000000,389000000,32000000,312000000,30000000,10000000,-118000000,2977000,True,100000.0,True,1112683520000,33163184.60
1,1,Arménie,2513,Orge,2013,vegetable,137000000,26000000,0,0.00,0.00000,0.00000,198000000,0,9000000,0,15000000,189000000,14000000,7000000,0,2977000,True,100000.0,True,0,0.00
2,1,Arménie,2514,Maïs,2013,vegetable,96000000,0,0,0.03,0.00000,0.00365,102000000,0,82000000,0,7000000,21000000,0,0,0,2977000,True,100000.0,True,0,10866.05
3,1,Arménie,2515,Seigle,2013,vegetable,1000000,0,365,0.12,0.00000,0.00730,1000000,0,0,0,0,1000000,0,0,0,2977000,True,100000.0,True,1086605000,21732.10
4,1,Arménie,2516,Avoine,2013,vegetable,4000000,0,730,0.37,0.01095,0.03285,6000000,0,1000000,1000000,0,5000000,0,0,0,2977000,True,100000.0,True,2173210000,97794.45
5,1,Arménie,2517,Millet,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000,True,100000.0,True,0,0.00
6,1,Arménie,2518,Sorgho,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000,True,100000.0,True,0,0.00
7,1,Arménie,2520,"Céréales, Autres",2013,vegetable,1000000,12000000,5110,1.96,0.01460,0.14235,21000000,0,4000000,6000000,1000000,17000000,1000000,0,0,2977000,True,100000.0,True,15212470000,423775.95
8,1,Arménie,2531,Pommes de Terre,2013,vegetable,225000000,16000000,29930,45.10,0.04380,0.70810,672000000,9000000,5000000,134000000,96000000,661000000,123000000,78000000,15000000,2977000,False,100000.0,True,89101610000,2108013.70
9,1,Arménie,2532,Manioc,2013,vegetable,0,0,0,0.00,0.00000,0.00000,0,0,0,0,0,0,0,0,0,2977000,False,100000.0,True,0,0.00


## Commentaires

Connaître la disponibilité intérieure en termes de kilo-calories et de protéines peut être utile si l'on souhaite connaître la disponibilité nutritive globale à l'échelle d'un pays ou d'une région.

___

# Part 4

## Objectifs

Calcul du ratio apport / poids exprimé en "kcal / kg" puis en "protéïnes (grammes) / kg" pour tous les produits.

___

## Méthode

Je prends en exemple trois produits dont je regarde les valeurs pour le monde, pour la France et je compare avec les valeurs trouvées sur Wikipédia.

___

## Code et résultats

In [14]:
main_df['total_food_weight'] = main_df['food_supply_kg_capita_year'] * main_df['pop_size']
main_df['total_food_weight'].fillna(0, inplace=True)

main_df['kcal_per_kg'] = main_df['total_food_supply'] / main_df['total_food_weight']
main_df['prot_per_kg'] = main_df['total_protein_supply'] / main_df['total_food_weight']

main_df['kcal_per_kg'].fillna(0, inplace=True)
main_df['prot_per_kg'].fillna(0, inplace=True)

main_df['kcal_per_kg'] = main_df['kcal_per_kg'].replace(np.inf, 0)
main_df['prot_per_kg'] = main_df['prot_per_kg'].replace(np.inf, 0)

main_df['kcal_per_kg'] = main_df['kcal_per_kg'].astype(int)

# World
working_df = main_df[main_df['total_food_supply'] > 0].copy()
working_df = working_df[working_df['total_protein_supply'] > 0].copy()
working_df = working_df[working_df['total_food_weight'] > 0].copy()

temp_df = working_df[working_df['product'] == 'Oeufs'].copy()
oeufs = to_int(temp_df['total_food_supply'].sum() / temp_df['total_food_weight'].sum())
oeufs_prot = to_int(temp_df['total_protein_supply'].sum() / temp_df['total_food_weight'].sum() * 1000)

temp_df = working_df[working_df['product'] == 'Bananes'].copy()
bananes = to_int(temp_df['total_food_supply'].sum() / temp_df['total_food_weight'].sum())
bananes_prot = to_int(temp_df['total_protein_supply'].sum() / temp_df['total_food_weight'].sum() * 1000)

temp_df = working_df[working_df['product'] == 'Dattes'].copy()
dattes = to_int(temp_df['total_food_supply'].sum() / temp_df['total_food_weight'].sum())
dattes_prot = to_int(temp_df['total_protein_supply'].sum() / temp_df['total_food_weight'].sum() * 1000)

# France
working_df = main_df[main_df['total_food_supply'] > 0].copy()
working_df = working_df[working_df['total_protein_supply'] > 0].copy()
working_df = working_df[working_df['total_food_weight'] > 0].copy()
working_df = working_df[working_df['country'] == 'France'].copy()

temp_df = working_df[working_df['product'] == 'Oeufs'].copy()
oeufs_fr = to_int(temp_df['total_food_supply'].sum() / temp_df['total_food_weight'].sum())
oeufs_prot_fr = to_int(temp_df['total_protein_supply'].sum() / temp_df['total_food_weight'].sum() * 1000)

temp_df = working_df[working_df['product'] == 'Bananes'].copy()
bananes_fr = to_int(temp_df['total_food_supply'].sum() / temp_df['total_food_weight'].sum())
bananes_prot_fr = to_int(temp_df['total_protein_supply'].sum() / temp_df['total_food_weight'].sum() * 1000)

temp_df = working_df[working_df['product'] == 'Dattes'].copy()
dattes_fr = to_int(temp_df['total_food_supply'].sum() / temp_df['total_food_weight'].sum())
dattes_prot_fr = to_int(temp_df['total_protein_supply'].sum() / temp_df['total_food_weight'].sum() * 1000)

print("Oeufs (kcal par kg) :")
print("-")
print(f"Monde : {oeufs}")
print(f"France : {oeufs_fr}")
print("Wikipédia : 1470")
print("-")

print("Bananes (kcal par kg) :")
print("-")
print(f"Monde : {bananes}")
print(f"France : {bananes_fr}")
print("Wikipédia : 723")
print("-")

print("Dattes (kcal par kg) :")
print("-")
print(f"Monde : {dattes}")
print(f"France : {dattes_fr}")
print("Wikipédia : 2820")
print("")
print("------------------------------------")
print("")

print("Oeufs (proteines en grammes par kg) :")
print("-")
print(f"Monde : {oeufs_prot}")
print(f"France : {oeufs_prot_fr}")
print("Wikipédia : 126")
print("-")

print("Bananes (proteines en grammes par kg) :")
print("-")
print(f"Monde : {bananes_prot}")
print(f"France : {bananes_prot_fr}")
print("Wikipédia : 9,8")
print("-")

print("Dattes (proteines en grammes par kg) :")
print("-")
print(f"Monde : {dattes_prot}")
print(f"France : {dattes_prot_fr}")
print("Wikipédia : 24,5")

Oeufs (kcal par kg) :
-
Monde : 1429
France : 1418
Wikipédia : 1470
-
Bananes (kcal par kg) :
-
Monde : 621
France : 629
Wikipédia : 723
-
Dattes (kcal par kg) :
-
Monde : 1621
France : 2808
Wikipédia : 2820

------------------------------------

Oeufs (proteines en grammes par kg) :
-
Monde : 111
France : 114
Wikipédia : 126
-
Bananes (proteines en grammes par kg) :
-
Monde : 8
France : 8
Wikipédia : 9,8
-
Dattes (proteines en grammes par kg) :
-
Monde : 15
France : 14
Wikipédia : 24,5


## Commentaires

Les valeurs peuvent être différentes en fonction des pays. Cette analyse permet de retrouver le ratio pour chaque couple pays / produit utilisé par la FAO.

___

# Part 5

## Objectifs

- Ciblage des 20 aliments les plus caloriques (ratio kcal / kg).
- Ciblage des 20 aliments les plus protéinés (ratio protéines / kg).

___

## Méthode

J'ai supprimé toutes les lignes ayant pour valeur 0 dans les colonnes ayant servi à calculer les ratios.

___


## Code et résultats

In [15]:
df_q5 = main_df.copy()
df_q5 = df_q5[df_q5['total_food_supply'] > 0].copy()
df_q5 = df_q5[df_q5['total_protein_supply'] > 0].copy()
df_q5 = df_q5[df_q5['total_food_weight'] > 0].copy()
df_q5 = df_q5[['product', 'kcal_per_kg', 'prot_per_kg']].copy()

df_q5 = df_q5.groupby(by='product').mean().copy()
df_q5.reset_index(inplace=True)
df_q5['kcal_per_kg'] = df_q5['kcal_per_kg'].astype(int)

df_q5.sort_values(by='kcal_per_kg', ascending=False, inplace=True)
print("Top kcal products")
display(df_q5[['product', 'kcal_per_kg']].iloc[0:20])
print("Top kcal products without oils")
display(df_q5[['product', 'kcal_per_kg']][df_q5['product'].str.contains('Huil') == False].iloc[0:20])

df_q5.sort_values(by='prot_per_kg', ascending=False, inplace=True)
print("Top proteins products")
display(df_q5[['product', 'prot_per_kg']].iloc[0:20])

Top kcal products


,product,kcal_per_kg
32,Huile d'Arachide,9125
35,Huile de Germe de Maïs,8859
30,Huil Plantes Oleif Autr,8854
40,Huile de Tournesol,8813
36,Huile de Palme,8654
38,Huile de Soja,8639
34,Huile de Colza&Moutarde,8567
31,Huile Graines de Coton,8539
33,Huile de Coco,8331
37,Huile de Palmistes,8311


Top kcal products without oils


,product,kcal_per_kg
28,Graisses Animales Crue,7560
9,"Beurre, Ghee",7282
78,Sésame,6136
5,Arachides Decortiquees,5705
23,Feve de Cacao,4905
61,"Plantes Oleiferes, Autre",4836
26,Graines Colza/Moutarde,4380
2,Aliments pour enfants,3826
25,Girofles,3721
71,Riz (Eq Blanchi),3640


Top proteins products


,product,prot_per_kg
26,Graines Colza/Moutarde,0.365000
73,Soja,0.315777
5,Arachides Decortiquees,0.250390
62,Pois,0.223035
44,Légumineuses Autres,0.221129
29,Haricots,0.218306
61,"Plantes Oleiferes, Autre",0.181831
78,Sésame,0.180210
85,"Viande, Autre",0.176298
0,Abats Comestible,0.176273


## Commentaires

Sans surprise, les huiles et les graisses sont les produits les plus caloriques. J'ai réalisé le même top 20 en excluant les huiles. On notera la présence dans ce second classement des légumineuses, du riz, des pois et des haricots.

Parmi les produits les plus protéinés, on retrouve également les légumineuses, la viande, le poisson et les oeufs. Les pois et haricots sont également en haut du classement. Ceci permet de mettre le doigt sur des produits à haut potentiel énergétique et qui sont potentiellement faciles et/ou économiques à produire.

___

# Part 6

## Objectifs

Calcul, pour les produits végétaux uniquement, de la disponibilité intérieure mondiale exprimée en kcal et en protéines.

___

## Méthode

J'ai sélectionné les données issues des produits végétaux grâce à la colonne "food_origin" que j'ai créée lors de la préparation des dataframes principaux.

___


## Code et résultats

In [16]:
main_df['domestic_supply_kcal'] = main_df['domestic_supply_kg'] * main_df['kcal_per_kg']
main_df['domestic_supply_prot'] = main_df['domestic_supply_kg'] * main_df['prot_per_kg']

veg_df = main_df[main_df['food_origin'] == 'vegetable'].copy()

world_veg_kcal = veg_df['domestic_supply_kcal'].sum()
world_veg_prot = veg_df['domestic_supply_prot'].sum()

print(f"""
World Vegetable Calories: {world_veg_kcal:,}"
World Vegetable Proteins: {world_veg_prot:,}"
""")


World Vegetable Calories: 12,163,402,106,000,000"
World Vegetable Proteins: 291,995,825,800.39294"



## Commentaires

Bien que les chiffres bruts semblent grands, il faut les confronter aux besoins nutritionnels des humains pour qu'ils prennent tout leur sens.

___

# Part 7

## Objectifs

Calcul du nombre d'humains que l'on pourrait nourrir avec la disponibilité intérieure mondiale en végétaux.

___

## Méthode

Je reprends les valeurs trouvées précédemment. Je divise par les données trouvées trouvées sur les sources suivantes :

### Besoins quotidiens en protéines

La RDA recommande 0,8 grammes de protéines par jour et par kilo.
Source : https://www.health.harvard.edu/blog/how-much-protein-do-you-need-every-day-201506188096

D'après la London School of Hygiene & Tropical Medicine, le poids moyen d'un humain est de : 62 kg
Source : https://www.telegraph.co.uk/news/earth/earthnews/9345086/The-worlds-fattest-countries-how-do-you-compare.html

Ce qui donne 62 * 0.8 = 49.6 grammes par jour et par humain.

### Besoins quotidiens en kilo-calories

Âge moyen d'un humain en 2013 : 40,6 ans
Source : https://www.insee.fr/fr/statistiques/2381476

Besoin en kilo-calories par jour à 40 ans : 2200 kcal / jour
Source : https://www.cnpp.usda.gov/sites/default/files/usda_food_patterns/EstimatedCalorieNeedsPerDayTable.pdf

___


## Code et résultats

In [17]:
pop_size = df_pop['pop_size'].sum()

kcal_humans = to_int(world_veg_kcal / cal_need)
prot_humans = to_int(world_veg_prot / prot_need)
kcal_percent = kcal_humans / pop_size
prot_percent = prot_humans / pop_size

print(f"We have enough calories to feed {kcal_humans:,} humans ({kcal_percent:.0%}).")
print(f"We have enough proteins to feed {prot_humans:,} humans ({prot_percent:.0%}).")

We have enough calories to feed 15,147,449,696 humans (216%).
We have enough proteins to feed 16,128,801,690 humans (230%).


## Commentaires

Avec les disponibilités intérieures mondiales de 2013, on pourrait nourrir le double d'humains présents sur terre, uniquement avec les produits végétaux ! Et ce tant en termes de protéines que de calories. Ceci met en évidence une piste parmi d'autres pour faire reculer la sous-nutrition. Au moins dans les pays où le climat permet l'agriculture des produits les plus nutritifs.

___

# Part 8

## Objectifs

Calcul du nombre d'humains qui pourraient être nourris avec :

- La disponibilité alimentaire des produits végétaux (Food)
- Les végétaux destinés à nourrir les animaux (Feed)
- Les pertes de produits végétaux (Waste)

___

## Méthode

- J'ai réalisé le calcul pour food + feed + losses.
- J'ai également réalisé la même procédure, uniquement avec "food".

___


## Code et résultats

In [18]:
df_q8 = main_df.copy()
df_q8 = df_q8[df_q8['food_origin'] == 'vegetable'].copy()
df_q8 = df_q8[['food_kg', 'feed_kg', 'losses_kg', 'kcal_per_kg', 'prot_per_kg']].copy()

# Total

df_q8['food_feed_losses_kg'] = df_q8['food_kg'] + df_q8['feed_kg'] + df_q8['losses_kg']
df_q8['food_feed_losses_kcal'] = df_q8['food_feed_losses_kg'] * df_q8['kcal_per_kg']
df_q8['food_feed_losses_prot'] = df_q8['food_feed_losses_kg'] * df_q8['prot_per_kg']

food_feed_losses_kg = to_int(df_q8['food_feed_losses_kg'].sum())
food_feed_losses_kcal = to_int(df_q8['food_feed_losses_kcal'].sum())
food_feed_losses_prot = to_int(df_q8['food_feed_losses_prot'].sum())

results = pd.DataFrame(columns=['Kcal', 'Proteins', 'Kcal for humans', 'Proteins for humans'])

kcal_for_humans = f"{food_feed_losses_kcal / cal_need:.0} ({food_feed_losses_kcal / cal_need / pop_size:.0%})"
prot_for_humans = f"{food_feed_losses_prot / prot_need:.0} ({food_feed_losses_prot / prot_need / pop_size:.0%})"

results = results.append({
    'Kcal': food_feed_losses_kcal,
    'Proteins': food_feed_losses_prot,
    'Kcal for humans': kcal_for_humans,
    'Proteins for humans': prot_for_humans
}, ignore_index=True)

print("Food + feed + losses supply: ")
display(results)

# Just food supply

df_q8 = main_df.copy()
df_q8 = df_q8[df_q8['food_origin'] == 'vegetable'].copy()
df_q8 = df_q8[['total_food_weight', 'total_food_supply', 'total_protein_supply']].copy()

food_feed_losses_kg = to_int(df_q8['total_food_weight'].sum())
food_feed_losses_kcal = to_int(df_q8['total_food_supply'].sum())
food_feed_losses_prot = to_int(df_q8['total_protein_supply'].sum())

results = pd.DataFrame(columns=['Kcal', 'Proteins', 'Kcal for humans', 'Proteins for humans'])

kcal_for_humans = f"{food_feed_losses_kcal / cal_need:.0} ({food_feed_losses_kcal / cal_need / pop_size:.0%})"
prot_for_humans = f"{food_feed_losses_prot / prot_need:.0} ({food_feed_losses_prot / prot_need / pop_size:.0%})"

results = results.append({
    'Kcal': food_feed_losses_kcal,
    'Proteins': food_feed_losses_prot,
    'Kcal for humans': kcal_for_humans,
    'Proteins for humans': prot_for_humans
}, ignore_index=True)

print("Food supply: ")
display(results)

Food + feed + losses supply: 


,Kcal,Proteins,Kcal for humans,Proteins for humans
0,9167530411000000,204375134675,1e+10 (163%),1e+10 (161%)


Food supply: 


,Kcal,Proteins,Kcal for humans,Proteins for humans
0,6051609427075000,125201642515,8e+09 (108%),7e+09 (99%)


## Commentaires

La disponibilité en nourriture + la nourriture pour bétail + les pertes permet de couvrir 150% des besoins alimentaires en termes de calories et de protéines dans le monde, simplement en regardant les produits végétaux !

On peut également constater que la "simple" disponibilité alimentaire en produits végétaux permet déjà de couvrir à elle seule 100% des besoins nutritionnels dans le monde !

___

# Part 9

## Objectifs

- Calcul de la disponibilité alimentaire mondiale.
- Expression en kcal, protéines puis pourcentage d'humains que l'on peut nourrir.

___

## Méthode

Au delà du calcul de l'apport nutritionnel de la disponibilité alimentaire mondiale, j'ai également calculé la partie représentée par les produits d'origine animale et la partie représentée par les produits d'origine végétale (également calculée dans la partie précédente).

___


## Code et résultats

In [19]:
df_q9 = main_df.copy()

print("""
All food
""")
world_cal = df_q9['total_food_supply'].sum()
world_prot = df_q9['total_protein_supply'].sum()

world_cal_humans = to_int(world_cal / cal_need)
world_prot_humans = to_int(world_prot / prot_need)

world_cal_humans_percent = world_cal_humans / pop_size 
world_prot_humans_percent = world_prot_humans / pop_size

print(f"We have enough food supply calories to feed {world_cal_humans:,} humans ({world_cal_humans_percent:.0%}).")
print(f"We have enough food supply proteines to feed {world_prot_humans:,} humans ({world_prot_humans_percent:.0%}).")

df_q9 = main_df[main_df['food_origin'] == 'vegetable'].copy()

print("""
Vegetable food
""")
world_cal = df_q9['total_food_supply'].sum()
world_prot = df_q9['total_protein_supply'].sum()

world_cal_humans = to_int(world_cal / cal_need)
world_prot_humans = to_int(world_prot / prot_need)

world_cal_humans_percent = world_cal_humans / pop_size 
world_prot_humans_percent = world_prot_humans / pop_size

print(f"We have enough food supply calories to feed {world_cal_humans:,} humans ({world_cal_humans_percent:.0%}).")
print(f"We have enough food supply proteines to feed {world_prot_humans:,} humans ({world_prot_humans_percent:.0%}).")

df_q9 = main_df[main_df['food_origin'] != 'vegetable'].copy()

print("""
Animal food
""")
world_cal = df_q9['total_food_supply'].sum()
world_prot = df_q9['total_protein_supply'].sum()

world_cal_humans = to_int(world_cal / cal_need)
world_prot_humans = to_int(world_prot / prot_need)

world_cal_humans_percent = world_cal_humans / pop_size 
world_prot_humans_percent = world_prot_humans / pop_size

print(f"We have enough food supply calories to feed {world_cal_humans:,} humans ({world_cal_humans_percent:.0%}).")
print(f"We have enough food supply proteines to feed {world_prot_humans:,} humans ({world_prot_humans_percent:.0%}).")


All food

We have enough food supply calories to feed 9,170,465,031 humans (131%).
We have enough food supply proteines to feed 11,448,253,429 humans (164%).

Vegetable food

We have enough food supply calories to feed 7,536,250,843 humans (108%).
We have enough food supply proteines to feed 6,915,689,489 humans (99%).

Animal food

We have enough food supply calories to feed 1,634,214,188 humans (23%).
We have enough food supply proteines to feed 4,532,563,940 humans (65%).


## Commentaires

On a théoriquement assez de disponibilité alimentaire pour nourrir plus que la terre entière en termes caloriques. Et en termes de protéines, on a une disponibilité alimentaire qui permet de couvrir 164% des besoins mondiaux (dont 100% uniquement avec les produits végétaux !).

Au vu des 800 000 personnes en sous-nutrition, cela montre une mauvaise répartition de la nourriture dans le monde.

On peut également s'interroger sur une production des produits protéinés bien trop élevée, qui aurait peut-être pour but de compenser cette mauvaise gestion. La nourriture d'origine animale pourrait être drastiquement réduite dans de meilleures conditions.

___

# Part 10

## Objectifs

Calcul de la population mondiale qui est considérée comme étant en sous-nutrititon.

___

## Méthode

Je récupère les données communiquées par la FAO. Le pourcentage est donné en fonction du nombre d'humains sur terre à la même année, en l'occurrence en 2013.

___


## Code et résultats

In [20]:
df_under = df_undernourishment.copy()
df_under.fillna(0, inplace=True)
df_under.sort_values(by='undernourishment', ascending=False, inplace=True)

total_under = to_int(df_under['undernourishment'].sum())
under_percent = total_under / pop_size
print(f"People undernourished: {total_under:,} ({under_percent:.1%})")

People undernourished: 739,699,999 (10.6%)


## Commentaires

Tous les chiffres de disponibilité alimentaire calculés précédemment montrent des sur-productions et des sur-abondances à l'échelle du monde, en termes d'apports nutritionnels.

Et pourtant, on constate que 10% de la population mondiale souffre de mal-nutrition. Cela pointe un problème de gestion de flux évident.

___

# Part 11

## Objectifs

Calcul du proportion des céréales destinées à **l'alimentation animale** (vs les céréales destinées à l'alimentation **des humains et des animaux**).

___

## Méthode

J'ai conservé uniquement les individus dont les variables "food_kg" et "feed_kg" ne contiennent pas de 0.

___


## Code et résultats

In [21]:
cereal_food = main_df[main_df['is_cereal'] == True].copy()
cereal_food = cereal_food[cereal_food['food_kg'] > 0]
cereal_food = cereal_food[cereal_food['feed_kg'] > 0]

animal_food = to_int(cereal_food['feed_kg'].sum())
human_food = to_int(cereal_food['food_kg'].sum())
ah_food = animal_food + human_food

print(f"Animal food: {animal_food:,} ({animal_food / ah_food:.0%})")
print(f"Human food: {human_food:,} ({human_food / ah_food:.0%})")
print(f"Total food: {animal_food + human_food:,} (100%)")

Animal food: 824,814,000,000 (47%)
Human food: 932,820,000,000 (53%)
Total food: 1,757,634,000,000 (100%)


## Commentaires

**La moitié de la production mondiale de céréales** est dédiée à nourrir le bétail ! Si l'on diminuait la production de nourriture d'origine animale, on pourrait instantanément libérer une partie de la production de céréales dédiée au bétail pour la dédier à la nourriture pour les humains.

___

# Part 12 - Preparation

## Objectif

1. Ciblage des pays dont une partie de la population est en sous-alimentation
2. Ciblage des 15 produits les plus exportés par ce groupe de pays en 2013
3. Ciblage des 200 plus grandes importations en 2013 pour ces 15 produits
4. Groupage de ces importations par produit

Pour ces 15 produits, calcul des ratios suivants :

1. Ratio Other Uses / Domestic Supply
2. Ratio Feed / Food + Feed

___

## Méthode

Le nouveau dataframe construit et ses deux colonnes 'other_vs_supply' et 'feed_vs_foodfeed' permettront d'atteindre les objectifs fixés dans la seconde partie du Part 12.

___


## Code et résultats

In [22]:
# Dataframe with just undernourishment countries
main_df_un = main_df[main_df['is_undernourishment'] == True].copy()

# Top 15 Export products
top_export_products = main_df_un.sort_values(by='export_quantity_kg', ascending=False)[['product_code', 'product']].drop_duplicates()[0:15]
top_export_products_name = [x for x in top_export_products['product']]
top_export_products_code = [x for x in top_export_products['product_code']]

# Print them
print("""Top 15 export products by undernourished countries :

""")
for i, product in enumerate(top_export_products_name):
    print(f"{i+1} - {product}")
    
# Filter - Only "Top 15 export products by poor countries" rows 
top_import_of_poor_exports = main_df.loc[(main_df['product_code'].isin(top_export_products_code)), :]

# Just the top 200 imports
top_import_of_poor_exports = top_import_of_poor_exports.sort_values(by='import_quantity_kg', ascending=False)[0:200]

final_columns = ['product_code', 'product', 'feed_kg', 'other_uses_kg', 'domestic_supply_kg', 'food_kg']
df_p11 = top_import_of_poor_exports[final_columns].copy()

df_p11 = df_p11.groupby(['product_code', 'product']).sum()
df_p11.reset_index(inplace=True)

# The two asked columns
df_p11['other_vs_supply'] = df_p11['other_uses_kg'] / df_p11['domestic_supply_kg']
df_p11['feed_vs_foodfeed'] = df_p11['feed_kg'] / (df_p11['feed_kg'] + df_p11['food_kg'])

df_p11.sort_values(by='feed_vs_foodfeed', ascending=False, inplace=True)

display(df_p11)

Top 15 export products by undernourished countries :


1 - Manioc
2 - Huile de Palme
3 - Maïs
4 - Riz (Eq Blanchi)
5 - Légumes, Autres
6 - Soja
7 - Blé
8 - Sucre Eq Brut
9 - Bananes
10 - Poissons Pelagiques
11 - Huile de Soja
12 - Orge
13 - Tomates
14 - Pommes
15 - Lait - Excl Beurre


,product_code,product,feed_kg,other_uses_kg,domestic_supply_kg,food_kg,other_vs_supply,feed_vs_foodfeed
1,2513,Orge,20153000000,12000000,29473000000,338000000,0.000407,0.983505
2,2514,Maïs,402885000000,182117000000,701068000000,55918000000,0.259771,0.878122
12,2763,Poissons Pelagiques,12156000000,0,16094000000,3937000000,0.000000,0.755359
5,2555,Soja,13710000000,524000000,159570000000,7510000000,0.003284,0.646089
3,2532,Manioc,25149000000,20355000000,67326000000,15257000000,0.302335,0.622408
0,2511,Blé,98715000000,15107000000,459892000000,303305000000,0.032849,0.245547
13,2805,Riz (Eq Blanchi),12644000000,2804000000,150234000000,122832000000,0.018664,0.093330
14,2848,Lait - Excl Beurre,30502000000,12659000000,379263000000,331684000000,0.033378,0.084216
9,2605,"Légumes, Autres",3980000000,0,87101000000,75711000000,0.000000,0.049943
4,2542,Sucre Eq Brut,0,2996000000,78453000000,64971000000,0.038188,0.000000


## Commentaires

Les pays dont une partie de la population est en sous-alimentation exportent des produits à haute valeur nutritive. Pommes, soja, tomates, bananes... C'est autant de nourriture qui pourrait être conservée sur le territoire pour nourrir la population qui en a besoin.

Environ un tiers des produits les plus exportés sont des produits qui servent ensuite à nourrir le bétail et qui ne sont pas destinés à nourrir les humains.

Environ un cinquième des produits les plus exportés sont des produits servant à d'autre usage que celui de nourrir des humains ou le bétail.

___

# Part 12

## Objectif

- Affichage des trois produits ayant le plus grand ratio Other Uses / Domestic Supply
- Affichage des trois produits ayant le plus grand ratio Feed / Food + Feed

___

## Méthode

Dans la précédente cellule, nous avons construit un dataframe (df_p11) contenant les produits les plus importés en provenance des pays en sous-nutrition. Ici nous allons simplement extraire le top 3 descendant sur les deux colonnes construites "Other Uses / Domestic Supply" et "Feed / Food + Feed".

___


## Code et résultats

In [23]:
print("""
Top 3 Other Uses / Domestic Supply:
""")

df_p11.sort_values(by='other_vs_supply', ascending=False, inplace=True)
for i in range(0,3):
    print(f"{i+1} - {df_p11['product'].iloc[i]} ({df_p11['other_vs_supply'].iloc[i]:.0%})")

print("""
Top 3 Feed / Food + Feed:
""")

df_p11.sort_values(by='feed_vs_foodfeed', ascending=False, inplace=True)
for i in range(0,3):
    print(f"{i+1} - {df_p11['product'].iloc[i]} ({df_p11['feed_vs_foodfeed'].iloc[i]:.0%})")


Top 3 Other Uses / Domestic Supply:

1 - Huile de Palme (72%)
2 - Huile de Soja (67%)
3 - Manioc (30%)

Top 3 Feed / Food + Feed:

1 - Orge (98%)
2 - Maïs (88%)
3 - Poissons Pelagiques (76%)


## Commentaires

Les produits exportés servant à nourrir les animaux sont **l'orge, le maïs et les poissons pélagiques**.

Les produits exportés ne servant ni à nourrir les animaux ni les hommes, sont **l'huile de palme, l'huile de soja et le manioc**. Les huiles sont très utilisés dans la cosmétique, les produits de soins et beauté. Ils servent entre autres à fabriquer du savon.

### Le manioc ne devrait pas être autant exporté par les pays dont la population est affamée

Bien que pauvre en protéines, le manioc est un aliment très calorique, qui peut combler une partie des apports nutritifs journaliers. Or on constate qu'il fait partie des produits les plus exportés par les pays dont une partie de la population souffre de malnutrition.

Des solutions tendent à être mises en place, notamment dans certains pays d'Afrique.

Sources :

- *Burkina Faso : diversifier l’usage du manioc* : https://www.afrik.com/burkina-faso-diversifier-l-usage-du-manioc
- *Le manioc : un aliment pour tout le monde* : http://www.fao.org/wairdocs/x5695f/x5695f03.htm
___

# Part 13

## Objectifs

Déterminer la quantité de céréales (en tonnes) libérées, si les Etats-Unis diminuaient de 10% leur production de produits animaux.

___


## Méthode

Je récupère sur le dataframe principal, uniquement les individus liés aux US. Je filtre ensuite uniquement sur les produits céréaliers. Enfin, je somme la variable "feed_kg".

J'ai également calculé ce que cela représente en termes d'apport nutritif pour les humains.

___


## Code et résulats

In [24]:
q13_df = main_df[main_df['country'] == "États-Unis d'Amérique"].copy()
q13_df = q13_df[q13_df['is_cereal'] == True].copy()

cereals_eaten_by_animals = to_int(q13_df['feed_kg'].sum())

print(f"10% equals: {to_int(cereals_eaten_by_animals * 0.1 / 1000):,} tons")

q13_df['feedkcal'] = q13_df['kcal_per_kg'] * q13_df['feed_kg']
q13_df['feedprot'] = q13_df['prot_per_kg'] * q13_df['feed_kg']

feedkcal = q13_df['feedkcal'].sum()
feedprot = q13_df['feedprot'].sum()

print(" ")
print(round(feedkcal * 0.1 / cal_need))
print(round(feedprot * 0.1 / prot_need))

10% equals: 14,009,600 tons
 
47897363.0
41026029.0


## Commentaires


Cela représenterait 14 millions de tonnes de céréales libérées ! On peut imaginer qu'une partie de cette disponibilité céréalière pourrait servir à nourrir des humains.

___

# Part 14

## Objectifs

Analyser pour la Thaïlande :

- La proportion du manioc exportée
- La proportion de personnes en sous-nutrition

___

## Méthode

Je récupère les informations à partir du dataframe principal.

___


## Code et résultats

In [25]:
q14_df = main_df[main_df['country'] == 'Thaïlande'].copy()
q14_df = q14_df[q14_df['product'] == 'Manioc'].copy()

manioc_exported = to_int(q14_df['export_quantity_kg'].iloc[0])
manioc_producted = to_int(q14_df['production_kg'].iloc[0])

print(f"Exported manioc: {to_int(manioc_exported / 1000):,} tons ({manioc_exported / manioc_producted:.0%})")

thai_pop = to_int(main_df[main_df['country'] == 'Thaïlande'].iloc[0]['pop_size'])
thai_un = to_int(df_undernourishment[df_undernourishment['country'] == 'Thaïlande']['undernourishment'].iloc[0])

print(f"People undernourished in Thaïland: {thai_un:,} people ({thai_un / thai_pop:.0%} of the population).")

Exported manioc: 25,214,000 tons (83%)
People undernourished in Thaïland: 6,100,000 people (9% of the population).


## Commentaires

Environ 10% de la population en Thaïlande est sous-alimentée. Or l'export de manioc représente 25 millions de tonnes ! Comme on l'a vu dans la partie 12, le manioc peut être une source d'alimentation non négligeable.

Aussi, limiter l'export pour en garder une partie sur le territoire permettrait potentiellement de diminuer la malnutrition dans le pays.

___

## Huile de palme et maïs exportés par les pays pauvres

In [26]:
train_df = main_df[main_df.is_undernourishment == True].copy()

# Palm Oil
po_df = train_df[train_df.product_code == 2577].copy()
print(f"{po_df['export_quantity_kg'].sum():,}")

# Maize
maize_df = train_df[train_df.product_code == 2514].copy()
print(f"{maize_df['export_quantity_kg'].sum():,}")

46,285,000,000
37,888,000,000


## Combien de personnes pourrait-on nourrir avec le Manioc exporté de Thaïlande

In [27]:
manioc_df = main_df[main_df['product'] == 'Manioc'].copy()
manioc_df = manioc_df[manioc_df['country'] == 'Thaïlande'].copy()

manioc_cal = manioc_df['export_quantity_kg'] * manioc_df['kcal_per_kg']
manioc_cal = manioc_cal / cal_need
manioc_cal = round(manioc_cal.iloc[0])
print(manioc_cal)

35261920.0
